
# **EAI 6010 Applications of Artifical Intelligence**

## **Week 3 - Machine Learning with XGboost**

Welcome to this week's session, where we will learn how to use XGBoost to create powerful prediction models using gradient boosting. Using Jupyter Notebooks you will learn how to create, evaluate, and tune XGBoost models efficiently. 

You will learn how to:

- How to instantiate and customize XGBoost models
- How to use XGBoost's DMatrix to optimize performance
- How to evaluate models in XGBoost using the right metrics
- How to tune parameters in XGBoost to achieve the best results
- How to visualize trees in XGBoost to analyze feature importance


## **The Dataset**

The session's dataset is a CSV file named `bookings.csv`, which contains data on hotel bookings. 

### **Acknowledgements**
The dataset was downloaded on [Kaggle](https://www.kaggle.com/jessemostipak/hotel-booking-demand/). The data is originally from an article called [Hotel booking demand datasets](https://www.sciencedirect.com/science/article/pii/S2352340918315191) by Nuno Antonio, Ana de Almeida, and Luis Nunes. It was then cleaned by Thomas Mock and Antoine Bichat for [#TidyTuesday during the week of February 11th, 2020](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-11/readme.md). For the purposes of this live training, it was further pre-processed to have cleaner ready-to-use features (e.g., dropping irrelevant columns, one-hot-encoding). The dataset has the following [license](https://creativecommons.org/licenses/by/4.0/).

### **Data Dictionary**

It contains the **53 columns**:

_For binary variables: `1` = true and `0` = false._

#### **Target** 
- `is_cancelled`: Binary variable indicating whether a booking was canceled

#### **Features** 
- `lead time`: Number of days between booking date and arrival date
- `arrival_date_week_number`, `arrival_date_day_of_month`, `arrival_date_month`: Week number, day date, and month number of arrival date 
- `stays_in_weekend_nights`, `stays_in_week_nights`: Number of weekend nights (Saturday and Sunday) and weeknights (Monday to Friday) the customer booked
- `adults`,`children`,`babies`: Number of adults, children, babies booked for the stay
- `is_repeated_guest`: Binary variable indicating whether the customer was a repeat guest 
- `previous_cancellations`: Number of prior bookings that were canceled by the customer
- `previous_bookings_not_canceled`: Number of prior bookings that were not canceled by the customer
- `required_car_parking_spaces`: Number of parking spaces requested by the customer
- `total_of_special_requests`: Number of special requests made by the customer
- `avg_daily_rate`: Average daily rate, as defined by dividing the sum of all lodging transactions by the total number of staying nights
- `booked_by_company`: Binary variable indicating whether a company booked the booking
- `booked_by_agent`: Binary variable indicating whether an agent booked the booking
- `hotel_City`: Binary variable indicating whether the booked hotel is a "City Hotel"
- `hotel_Resort`: Binary variable indicating whether the booked hotel is a "Resort Hotel"
- `meal_BB`: Binary variable indicating whether a bed & breakfast meal was booked 
- `meal_HB`: Binary variable indicating whether a half board meal was booked
- `meal_FB`: Binary variable indicating whether a full board meal was booked 
- `meal_No_meal`: Binary variable indicating whether there was no meal package booked 
- `market_segment_Aviation`, `market_segment_Complementary`, `market_segment_Corporate`, `market_segment_Direct`, `market_segment_Groups`, `market_segment_Offline_TA_TO`, `market_segment_Online_TA`, `market_segment_Undefined`: Indicates market segment designation with a value of `1`. "TA"= travel agent, "TO"= tour operators
- `distribution_channel_Corporate`, `distribution_channel_Direct`, `distribution_channel_GDS`, `distribution_channel_TA_TO`, `distribution_channel_Undefined`: Indicates booking distribution channel with a value of `1`. "TA"= travel agent, "TO"= tour operators, "GDS" = Global Distribution System
- `reserved_room_type_A`,`reserved_room_type_B`, `reserved_room_type_C`,`reserved_room_type_D`, `reserved_room_type_E`, `reserved_room_type_F`, `reserved_room_type_G`, `reserved_room_type_H`, `reserved_room_type_L`: Indicates code of room type reserved with a value of `1`. Code is presented instead of designation for anonymity reasons
- `deposit_type_No_Deposit`: Binary variable indicating whether a deposit was made
- `deposit_type_Non_Refund`: Binary variable indicating whether a deposit was made in the value of the total stay cost
- `deposit_type_Refundable`: Binary variable indicating whether a deposit was made with a value under the total stay cost 
- `customer_type_Contract`: Binary variable indicating whether the booking has an allotment or other type of contract associated to it 
- `customer_type_Group`: Binary variable indicating whether the booking is associated to a group 
- `customer_type_Transient`: Binary variable indicating whether the booking is not part of a group or contract, and is not associated to other transient booking
- `customer_type_Transient-Party`: Binary variable indicating whether the booking is transient, but is associated to at least another transient booking

---



## **1. Getting to know our data**

Let's get to know our columns and split our data into features and labels!

In [9]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Import libraries
import numpy as np
import pandas_profiling
from scipy.stats import mode
import scipy as sc
import pandas as pd
import xgboost as xgb # XGBoost typically uses the alias "xgb"
import numpy as np
from pandas import DataFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
!pip install graphviz

In [5]:
#bookings.to_csv(r'/Users/abhijitsanyal/Documents/Northeastern_Univ/NEU_AI_6010_Fall_2020/data/bookings.csv')

In [12]:
# Read in dataset
bookings= pd.read_csv('D:/Data/Northeastern University/Applications of AI/Module 3/bookings.csv', index_col = "Unnamed: 0")
pd.set_option('display.max_columns', None)


In [13]:
#Get the dimension of the dataset
bookings.shape
bookings.info()

(119210, 53)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119210 entries, 0 to 119209
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   is_canceled                     119210 non-null  int64  
 1   lead_time                       119210 non-null  int64  
 2   arrival_date_week_number        119210 non-null  int64  
 3   arrival_date_day_of_month       119210 non-null  int64  
 4   arrival_date_month              119210 non-null  int64  
 5   stays_in_weekend_nights         119210 non-null  int64  
 6   stays_in_week_nights            119210 non-null  int64  
 7   adults                          119210 non-null  int64  
 8   children                        119206 non-null  float64
 9   babies                          119210 non-null  int64  
 10  is_repeated_guest               119210 non-null  int64  
 11  previous_cancellations          119210 non-null  int64  
 12  previous_booking

It looks like we have 53 columns with 119,210 rows. All the datatypes are numeric and ready for use.

In [14]:
# Take a closer look at column distributions
bookings.describe()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,required_car_parking_spaces,total_of_special_requests,avg_daily_rate,booked_by_company,booked_by_agent,hotel_City,hotel_Resort,meal_BB,meal_FB,meal_HB,meal_No_meal,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline_TA_TO,market_segment_Online_TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA_TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,deposit_type_No_Deposit,deposit_type_Non_Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
count,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119206.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000,119210.000000
mean,0.370766,104.109227,27.163376,15.798717,6.552051,0.927053,2.499195,1.859206,0.104047,0.007961,0.031499,0.087191,0.137094,0.062553,0.571504,101.969092,0.056774,0.863434,0.664063,0.335937,0.773727,0.006694,0.121282,0.098297,0.001971,0.006107,0.044308,0.105545,0.166018,0.202852,0.473182,0.000017,0.055792,0.122565,0.001619,0.819982,0.000042,0.720351,0.009353,0.007810,0.160884,0.054685,0.024276,0.017549,0.005042,0.000050,0.876277,0.122364,0.001359,0.034158,0.004815,0.750575,0.210452
std,0.483012,106.875450,13.601107,8.781070,3.089796,0.995117,1.897106,0.575186,0.398842,0.097509,0.174663,0.844918,1.498137,0.245360,0.792876,50.434007,0.231411,0.343390,0.472319,0.472319,0.418420,0.081543,0.326456,0.297717,0.044356,0.077908,0.205780,0.307255,0.372098,0.402125,0.499282,0.004096,0.229521,0.327939,0.040204,0.384204,0.006476,0.448829,0.096259,0.088027,0.367426,0.227365,0.153907,0.131305,0.070825,0.007094,0.329266,0.327707,0.036839,0.181636,0.069223,0.432682,0.407631
min,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,18.000000,16.000000,8.000000,4.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.500000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,69.000000,28.000000,16.000000,7.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,94.950000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,

In [15]:
# Plot cancellation counts to visualize proportion of not cancelled and cancelled
bookings['is_canceled'].value_counts().plot(kind='bar')

<AxesSubplot:>

Remember for our binary variables, like `is_canceled`, `1` = true and `0` = false.

In [16]:
# The target variable is a binary variable and we first examine its distributio
# Get an exact percentage of not cancelled and cancelled
bookings['is_canceled'].value_counts()
bookings['is_canceled'].value_counts()/bookings['is_canceled'].count()*100

0    75011
1    44199
Name: is_canceled, dtype: int64

0    62.923412
1    37.076588
Name: is_canceled, dtype: float64

### **Which features are most correlated to cancelations?**

#### **Correlation Coefficient**
- Quantifies the linear relationship between two variables
- Number between -1 and 1
- Magnitude corresponds to strength of relationship
- Sign (+ or -) corresponds to direction of relationship
- Most common way to calculate: **Pearson product-moment correlation coefficient**

<br>

$${\large\displaystyle \rho _{X,Y}={\frac {\operatorname {cov} (X,Y)}{\sigma _{X}\sigma _{Y}}}}$$

<br>

We can use `pandas`'s [function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) `DataFrame.corr()` which returns a correlation matrix using the Pearson correlation coefficient as default. 

In [17]:
# Compute correlation matrix 
bookings.corr()
corr_bookings = DataFrame(bookings.corr())
# Check the dimensions of the correlation matrix
corr_bookings.shape
# Write out correlation matrix to Excel; Examine correlations in Excel with Conditional Formatting 
corr_bookings.to_excel('/Users/abhijitsanyal/Documents/Northeastern_Univ/NEU_AI_6010_Fall_2020_v2/data/corr_bookings.xlsx', sheet_name = 'corr_bookings', index= False)

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,required_car_parking_spaces,total_of_special_requests,avg_daily_rate,booked_by_company,booked_by_agent,hotel_City,hotel_Resort,meal_BB,meal_FB,meal_HB,meal_No_meal,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline_TA_TO,market_segment_Online_TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA_TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,deposit_type_No_Deposit,deposit_type_Non_Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
is_canceled,1.000000,0.292876,0.008315,-0.005948,0.011179,-0.001323,0.025542,0.058182,0.004862,-0.032569,-0.083745,0.110139,-0.057365,-0.195701,-0.234877,0.046492,-0.099692,0.102218,0.137082,-0.137082,0.012786,0.038790,-0.020085,-0.006571,-0.013755,-0.040330,-0.081645,-0.154366,0.221990,-0.028671,-0.006232,0.005336,-0.075589,-0.151583,-0.014928,0.176005,0.005755,0.069117,-0.008372,-0.007336,-0.047736,-0.038882,-0.021778,-0.001672,0.005436,-0.000550,-0.477957,0.481507,-0.011345,-0.023690,-0.038842,0.133235,-0.124271
lead_time,0.292876,1.000000,0.127046,0.002306,0.131603,0.085985,0.166892,0.117575,-0.037886,-0.021003,-0.123209,0.086025,-0.073599,-0.116624,-0.095949,-0.065018,-0.125951,0.179563,0.075970,-0.075970,-0.039154,0.009646,0.136377,-0.097156,-0.041433,-0.066418,-0.165143,-0.174242,0.346418,0.146264,-0.186607,-0.003933,-0.134400,-0.161542,-0.031422,0.221546,-0.004915,0.104115,0.008561,-0.019798,-0.070515,-0.029329,-0.052407,-0.029744,-0.017132,-0.006911,-0.380173,0.380120,0.016564,0.068627,-0.031759,-0.174030,0.159538
arrival_date_week_number,0.008315,0.127046,1.000000,0.066572,0.995101,0.018629,0.016047,0.026567,0.005559,0.010417,-0.031125,0.035493,-0.021009,0.001980,0.026202,0.076281,-0.025516,0.032239,0.001241,-0.001241,-0.000361,0.021425,0.038426,-0.047495,-0.006805,0.007846,-0.017845,-0.016997,0.002940,0.068223,-0.039955,0.001457,0.007821,-0.016003,-0.003981,0.009370,0.002017,0.008490,0.007617,0.005359,-0.010173,-0.007899,0.002586,-0.002047,0.005354,0.001393,-0.005903,0.007831,-0.016901,0.090342,0.011246,-0.079507,0.042228
arrival_date_day_of_month,-0.005948,0.002306,0.066572,1.000000,-0.026335,-0.016225,-0.028362,-0.001754,0.014541,-0.000235,-0.006471,-0.027027,-0.000306,0.008569,0.003026,0.030291,-0.002831,-0.003920,-0.001678,0.001678,0.008278,-0.002570,-0.005223,-0.005203,-0.000984,-0.002985,-0.002046,0.004075,0.011860,-0.011060,-0.000997,-0.005504,-0.013528,0.011903,-0.001144,-0.001839,-0.007080,-0.017273,0.007001,0.001328,0.006459,0.011450,0.007613,0.004861,0.002873,-0.004012,0.005003,-0.008643,0.032171,-0.012178,-0.001704,-0.000426,0.006168
arrival_date_month,0.011179,0.131603,0.995101,-0.026335,1.000000,0.018851,0.019739,0.029239,0.005483,0.010193,-0.031709,0.037473,-0.021745,0.000325,0.028086,0.079828,-0.026905,0.034048,0.001770,-0.001770,-0.002757,0.021823,0.040014,-0.045978,-0.006961,0.007879,-0.019697,-0.017274,0.002607,0.067672,-0.038325,0.001919,0.006841,-0.017270,-0.004291,0.011059,0.002616,0.008491,0.006274,0.006447,-0.009870,-0.008189,0.002405,-0.001879,0.005222,0.001794,-0.006471,0.008809,-0.020520,0.091687,0.011457,-0.079496,0.041581
stays_in_weekend_nights,-0.001323,0.085985,0.018629,-0.016225,0.018851,1.000000,0.494175,0.094759,0.046134,0.018607,-0.086009,-0.012769,-0.042859,-0.018520,0.073124,0.050670,-0.108566,0.127345,-0.187816,0.187816,-0.065954,0.017596,0.105888,-0.028236,0.007819,-0.044784,-0.107029,-0.024834,-0.06

(53, 53)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/abhijitsanyal/Documents/Northeastern_Univ/NEU_AI_6010_Fall_2020_v2/data/corr_bookings.xlsx'

In [18]:
# Which features are most correlated with is_cancelled?
bookings.corr()['is_canceled'].sort_values(ascending=False)


is_canceled                       1.000000
deposit_type_Non_Refund           0.481507
lead_time                         0.292876
market_segment_Groups             0.221990
distribution_channel_TA_TO        0.176005
hotel_City                        0.137082
customer_type_Transient           0.133235
previous_cancellations            0.110139
booked_by_agent                   0.102218
reserved_room_type_A              0.069117
adults                            0.058182
avg_daily_rate                    0.046492
meal_FB                           0.038790
stays_in_week_nights              0.025542
meal_BB                           0.012786
arrival_date_month                0.011179
arrival_date_week_number          0.008315
distribution_channel_Undefined    0.005755
reserved_room_type_H              0.005436
market_segment_Undefined          0.005336
children                          0.004862
reserved_room_type_L             -0.000550
stays_in_weekend_nights          -0.001323
reserved_ro

Correlation does not imply causation and does not priortize feature importance! Correlation may not necessarily help differentiate classes. Also, the Pearson coefficient only considers linear relationships, and some of these variables are binary for which linear relationships may not be meaningful. 

### **Splitting data**
Let's split our label and features so we can get to building models! The first column is our target label `is_cancelled`. The rest are features.

In [19]:
# Define X and y
X, y = bookings.iloc[:,1:], bookings.iloc[:,0]

---
<center><h1> XGBoost Classifier </center>

---

## **2. XGBoost Classifier**

XGBoost has a  [scikit-learn API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn), which is useful if you want to use different scikit-learn classes and methods on an XGBoost model (e.g.,`predict()`, `fit()`).  In this section, we will try the API out with the `xgboost.XGBClassifier()` class and get a baseline accuracy for the rest of our work. So that our results are reproducible, we will set the `random_state=1234`.

As a reminder, gradient boosting sequentially trains weak learners where each weak learner tries to correct its predecessor's mistakes. First, we will instantiate a simple XGBoost classifier without changing any of the other parameters, and we will inspect the parameters that we have not touched.

In [20]:
from sklearn.model_selection import train_test_split

# Train and test split using sklearn
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=.33, random_state=1234)

# Instatiate a XGBClassifier and call it xgb_clf
xgb_clf = xgb.XGBClassifier(random_state=123)

# Inspect the parameters
xgb_clf.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 123,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

A few things to note:
- The `booster` parameter is `gbtree`.  This means the weak learners, or boosters, are decision trees in this model. `gbtree` is the default, and we will keep it this way.
- The `objective` function, or loss function, is defined as `binary:logistic`. The objective function quantifies how far off a prediction is from the actual results. We want to minimize this to have the smallest possible loss. `binary:logistic` is the default for classifiers. `binary:logistic` outputs the actual predicted probability of the positive class (in our case, that a booking is cancelled).
- `n_estimators` is the number of gradient boosted trees we want in our model. It's equivalent to the number of boosting rounds. For our purposes, we don't want too many boosting rounds, or training will take too long. **Let's lower it from 100 to 10**.

<br>

- `max_depth` is the maximum tree depth allowed. Tree depth is the length of the longest path from the root node to a leaf node. Making this too high will give our model more variance, or more potential to overfit. Similar to `n_estimators`, the more we increase this, the longer our training period will be. **Let's keep this at 3**.


**If you are using XGBoost for a multi-class problem then you will need to tinker with the code given below:**

evals_result = {}

    self.classes_ = np.unique(y)
    
    self.n_classes_ = len(self.classes_)

    xgb_options = self.get_xgb_params()

    if callable(self.objective):
        obj = _objective_decorator(self.objective)
        # Use default value. Is it really not used ?
        xgb_options["objective"] = "binary:logistic"
    else:
        obj = None

    if self.n_classes_ > 2:
        # Switch to using a multiclass objective in the underlying XGB instance
        xgb_options["objective"] = "multi:softprob"
        xgb_options['num_class'] = self.n_classes_

In [21]:
## Set max_depth to be 10 instead of 100
xgb_clf.set_params(n_estimators=10)

# Fit it to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              objective='binary:logistic', random_state=123, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=True,
              validate_parameters=None, verbosity=None)

E:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:21:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=8, num_parallel_tree=1,
              objective='binary:logistic', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

We are going to use **accuracy** as our metric for this problem since we only have two classes, and the smallest class takes ~37% of the data. To calculate accuracy, we need to count the correctly predicted data points and divide it by the total number of data points.

In [22]:
# Calculate the accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

# Print the baseline accuracy
print("Baseline accuracy:", accuracy)

Baseline accuracy: 0.814616166751398


### **Visualizing your tree**

`XGBoost` has two handy visualization functions for interpreting results.

The first is `plot_importance()`  which plots feature importance, meaning, how predictive each feature is for the target variable. It takes in the fitted XGBoost model fitted.


#### **Plotting feature importance**

In [23]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 8)

xgb.plot_importance(xgb_clf, max_num_features = 15)

<AxesSubplot:title={'center':'Feature importance'}, xlabel='F score', ylabel='Features'>

How is importance calculated? Here's an excerpt from the [documentation](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.plotting):

> **importance_type (str, default "weight") –**
> How the importance is calculated: either “weight”, “gain”, or “cover”
> - ”weight” is the number of times a feature appears in a tree
> - ”gain” is the average gain of splits which use the feature
> - ”cover” is the average coverage of splits which use the feature where coverage is defined as the number of samples affected by the split

We'll focus on the first two. Gain is way to quantify how much a feature contributes to improving accuracy. Let's try out `gain` to see how it differs from `weight`.

In [24]:
# Plot gain instead of weight
xgb.plot_importance(xgb_clf, importance_type="gain")

<AxesSubplot:title={'center':'Feature importance'}, xlabel='F score', ylabel='Features'>

In [20]:
conda install python-graphviz

Solving environment: ...working... done

## Package Plan ##

  environment location: E:\Anaconda

  added / updated specs:
    - python-graphviz


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py37haa95532_0         2.9 MB
    graphviz-2.38              |       hfd603c8_2        29.3 MB
    python-graphviz-0.16       |     pyhd3eb1b0_1          20 KB
    ------------------------------------------------------------
                                           Total:        32.2 MB

The following NEW packages will be INSTALLED:

  graphviz           pkgs/main/win-64::graphviz-2.38-hfd603c8_2
  python-graphviz    pkgs/main/noarch::python-graphviz-0.16-pyhd3eb1b0_1

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.9.2-py37h03978a9~ --> pkgs/main::conda-4.9.2-py37haa95532_0




conda-4.9.2        

In [25]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [26]:
conda install python-graphviz

Solving environment: ...working... done

# All requested packages already installed.

Failed to create temp directory "C:\Users\Shivani Adsar\AppData\Local\Temp\conda-<RANDOM>\" 

Note: you may need to restart the kernel to use updated packages.


In [27]:
import matplotlib
import matplotlib.pyplot as plt

In [28]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [29]:
import xgboost as xgb
xgb_regressor = xgb.XGBRegressor()

In [30]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

Here we see the ordering of features differs quite a bit between `gain` and `weight`! This implies that (1) feature importance can be subjective, and (2) the number of appearances a feature has on a tree is not necessarily correlated to how much gain it brings. For example, a binary variable has less of a chance to appear as many times as a continuous variable on a tree, since there are only two outputs. However, it can still be a powerful feature.

#### **Plotting a decision tree from your model**

The second handy visualization function is `plot_tree()` which visually generates a decision tree in your model. Remember, there are multiple trees in the model.

It takes in the fitted XGBoost model, and with the `num_trees` parameter, you can indicate which tree you want to see. For example, if I have `n_estimator=3`, I can set the `num_trees` parameter as `0`, `1`, or `2` to see the first, second, or third tree, respectively.

If you get a runtime error as this then check the following stackoverflow entry:
https://stackoverflow.com/questions/35064304/runtimeerror-make-sure-the-graphviz-executables-are-on-your-systems-path-aft

I had to run `conda install python-graphviz` since I use a Mac with Anaconda

In [31]:
matplotlib.rcParams['figure.figsize'] = (20.0, 8)

# Plot the first tree
xgb.plot_importance(xgb_clf)
xgb.plot_tree(xgb_clf, num_trees=0)

<AxesSubplot:title={'center':'Feature importance'}, xlabel='F score', ylabel='Features'>

<AxesSubplot:>

The leaf output is the raw score. It's converted to a probability at the end of training.

There's a parameter called `rankdir`. We can set it to `LR` so the decision tree builds left to right, as opposed to top to bottom.

In [32]:
# Plot the last tree
xgb.plot_tree(xgb_clf, num_trees=9, rankdir='LR')

<AxesSubplot:>

---
<center><h1> Cross Validation </h1> </center>

---

## **3. Cross Validation in XGBoost**

Cross validation is considered best practice for assessing a model's performance. We can use `xgboost.cv()` to efficiently run cross validation on XGBoost models. This method is part of XGBoost's core library and **not** part of XGBoost's scikit-learn API from earlier. 


### **Converting to DMartix**

**DMatrix is XGBoost's internal data structure designed to optimize for both memory efficiency and training speed.** It's one of the reasons XGBoost achieves performance and efficiency gains over other implementations of gradient boosting.

In the previous exercise, the inputted datasets were converted into DMatrix data when we fit the data with `.fit()`. However, to use the `xgboost.cv()` object, we need to first explicitly convert the data into a `DMatrix`.

In [33]:
# Convert X and y into a DMatrix
bookings_dmatrix = xgb.DMatrix(data=X,label=y)

### **Cross validation with xgb.cv**

Note: Scikit-learn uses `n_estimator` to refer to the number of boosting rounds, or the number of trees in the model. In XGBoost, it's referred to as `num_boost_rounds`.

To define the booster parameters, it's common practice to create a dictionary to hold all the parameters related to the individual boosters. We'll do this and define the objective functions as `binary:logistic` and the maximum tree depth to be `3`.

In [34]:
# Define booster parameters using a dictionary
params = {"objective":"binary:logistic", 'max_depth': 3}

# Instantiate a CV object with 3 folds and 10 boosting rounds
xgb_cv = xgb.cv(dtrain=bookings_dmatrix, params=params, nfold=3, num_boost_round=10, seed=123)

# Inspect the results: how are they stored?
xgb_cv

[22:23:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:23:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:23:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.605010,0.000372,0.605403,0.000401
1,0.548085,0.002531,0.548340,0.002941
2,0.514772,0.000613,0.515019,0.001056
3,0.492943,0.000655,0.493099,0.000761
4,0.477932,0.000571,0.478055,0.000689
5,0.465592,0.000987,0.465832,0.001177
6,0.454656,0.000433,0.454879,0.001785
7,0.447301,0.000500,0.447635,0.001840
8,0.440716,0.001202,0.441092,0.002190
9,0.434995,0.000723,0.435372,0.000988


#### **Results in XGBoost**

There are ten rows for our ten boosting rows.

`error` as defined by [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters):
> Binary classification error rate. It is calculated as `#(wrong cases)/#(all cases)`. For the predictions, the evaluation will regard the instances with prediction value larger than 0.5 as positive instances, and the others as negative instances.

This means we need to subtract the last boosting round's `test-error-mean` from 1 to get the accuracy.

In [35]:
# Caclulate accuracy
accuracy= 1 - xgb_cv["test-error-mean"].iloc[-1]

# Print the baseline accuracy
print("baseline cv accuracy:", accuracy)

KeyError: 'test-error-mean'

Let's now look into improving performace.

### **More trees with early stopping**

We've been using ten trees (aka `num_boost_rounds`). Let's add more trees (40 instead of 10), but make sure to add **early stopping**. 

Early stopping works by testing the model after every boosting round against the holdout set. If the holdout metric (error in our case) has not improved after a given number of rounds (defined by `early_stopping_rounds`), then any additional boosting rounds are stopped. If the model continuously improves up to `num_boost_round`, then early stopping does not occur.

This helps automatically select the number of boosting rounds and minimize additional training time.

In [46]:
# Increase the number of trees to 40 and set the early stopping rounds to 10
xgb_cv = xgb.cv(dtrain=bookings_dmatrix, params=params, nfold=3,
                    num_boost_round=40,early_stopping_rounds=10, seed=123)

# Caclulate accuracy
accuracy= 1 - xgb_cv["test-error-mean"].iloc[-1]

# Print the accuracy
print("accuracy:", accuracy)

accuracy: 0.8146966666666667


In [47]:
# Did early stopping happen?
xgb_cv

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.238533,0.006855,0.239460,0.009837
1,0.232766,0.001537,0.232942,0.000840
2,0.232849,0.001445,0.233001,0.000868
3,0.233126,0.001120,0.233194,0.001102
4,0.232812,0.000860,0.232866,0.001432
5,0.220007,0.018839,0.219394,0.017942
6,0.193465,0.000814,0.193541,0.002012
7,0.193394,0.000787,0.193591,0.001621
8,0.192970,0.000645,0.193096,0.002312
9,0.192035,0.000728,0.192081,0.001525


There are 40 rows, and the test error seems to have decreased almost continuously after each round. This means early stopping did not happen because our model kept improving.

We've improved the results by increasing the number of boosted trees, but there are more parameters we can play with!

## **4. Digging into Parameters**

Along with setting the number of boosting rounds and early stopping rounds, there are many other parameters for our tree booster. We can see this in the documentation for [parameters for tree booster](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster).

In this section, we'll take a look at several important parameters and understand what they do.

For this session's purpose, we'll use `XGBClassifier()` with **25 boosting** rounds to avoid long training times. 

In [49]:
from sklearn.metrics import accuracy_score

# Instantiate the XGBClassifier with 25 boosting rounds
xgb_clf = xgb.XGBClassifier(n_estimators=25, random_state=123)

### **Max depth**

_From XGBoost docs:_
> Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.


Let's see what happens when we increase the `max_depth` from 3 to 10.


In [51]:
# Set max_depth to 10
xgb_clf.set_params(max_depth=10)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

0.8254448398576513

### **colsample_bytree**

_From XGBoost docs:_

> The subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.

Essentially, this lets us limit the number of columns used when constructing each tree. This adds randomness, making the model more robust to noise. The default is 1 (i.e. all the columns), let's try a smaller value.


In [52]:
# Set colsample_bytree to 0.5 
xgb_clf.set_params(colsample_bytree=0.5)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

0.8298678190137265

We can also limit the number of columns used by every depth level or node of our tree.

_From XGBoost docs:_

> `colsample_bylevel` is the subsample ratio of columns for each level. Subsampling occurs once for every new depth level reached in a tree. Columns are subsampled from the set of columns chosen for the current tree.

> `colsample_bynode` is the subsample ratio of columns for each node (split). Subsampling occurs once every time a new split is evaluated. Columns are subsampled from the set of columns chosen for the current level.

### **subsample**

_From XGBoost docs:_

> - Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees and this will prevent overfitting. 
> - Subsampling will occur once in every boosting iteration.
> - range: (0,1]

Default is 1, let's try 0.75. 

This means each of our 25 trees will get a random sampling of 75% of our training data. Each tree will train on different portions of data which adds randomness (similar to `colsample_bytree`). 


However, we don't want this too low if we don't have many trees, because our model will underfit from not seeing as much data.

In [53]:
# Set subsample to 0.75 
xgb_clf.set_params(subsample=0.75)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

0.8293594306049822

### **gamma**

_From XGBoost docs:_
> - Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
> - range: [0,∞]


This decides whether a node will split based on the expected loss reduction loss after the split. `gamma` represents the minimum loss reduction required for a node to split.

Increasing `gamma` = less splits = less complexity

The default is 0, so in our case, nodes have always split until the maximum depth. Let's increase it to 0.25.




In [54]:
# Set gamma to .25 
xgb_clf.set_params(gamma=0.25)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

0.8291306558210473

### **Learning Rate (aka eta)**

_From XGBoost docs:_
> - Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
> - range: [0,1]

The learning rate affects how quickly a model learns.

Gradient boosting works by sequentially adding weak learners to the model. Each new weak learner attempts to correct the residual errors from the preceding trees. This make the model very susceptible to overfitting. Learning rate can help slow down learning by shrinking the resulting weights of the current tree before passing them on to the next tree.

The learning rate of our model is currently 0.1. What happens if we change it to 0.3?

In [55]:
# Set learning rate to .3 
xgb_clf.set_params(learning_rate=0.3)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25,
              learning_rate=0.3, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25,
              learning_rate=0.3, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

0.8385358413828164

The learning rate and the number of trees should be tuned together. If we decrease the learning rate, we need to make sure we have enough trees to learn something and avoid severely underfitting. **Therefore, a low learning will require more boosting rounds.**

### **reg_alpha**

_From XGBoost docs:_
> L1 regularization term on weights. Increasing this value will make model more conservative.

L1 is often referred to as **lasso regression**. It's a foundational regularization technique, meaning it aims to reduce overfitting by discouraging complex models. In the case of gradient boosting, L1 does this by adding penalties on leaf weights. Increasing `alpha` drives base learners' leaf weights towards 0.

Default is 0, meaning there is no alpha regularization in our model currently. Let's activate L1 with a value of `0.01`.

In [56]:
# set reg_alpha to .1 
xgb_clf.set_params(reg_alpha=0.01)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25,
              learning_rate=0.3, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.25,
              learning_rate=0.3, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=123,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.75, verbosity=1)

0.8388662938485002

**L2**, aka ridge regression, is also available with the parameter `reg_lambda`. L2 is known for having smoother penalty then L1. This means leaf weights smoothly decrease rather with less risk of sparsity in the leaf weights. So, make sure to try our different regularization techniques! 

Let's take a look at our resulting parameters after we manually changed them.

In [57]:
# Print the model parameters
xgb_clf.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.5,
 'gamma': 0.25,
 'learning_rate': 0.3,
 'max_delta_step': 0,
 'max_depth': 10,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 25,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 123,
 'reg_alpha': 0.01,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 0.75,
 'verbosity': 1}

There are a lot of possible parameters combinations. We can't manually tune and pick them.

---
<center><h1> Hyperparameter Tuning </center>

---

## **5. Hyperparameter tuning with Random Search**

Grid search and random search are the most popular methods for hyperparameter tuning. However, grid search can get computationally expensive if you are searching over a large hyperparameter space and dealing with multiple hyperparameters. In this last section, this is why we'll use random search because it doesn't try all the hyperparameter values. In random search, a fixed number of hyperparameter settings is sampled from specified probability distributions.

XGBoost doesn't have a built-in gridsearch function, so we need to use `scikit-learn`'s [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). This means we'll have to use `XGBClassifier()` because it's `scikit-learn` compatible.


<br>

The `RandomizedSearchCV()` function takes in the following arguments:

- `estimator`: The estimator being fit, here it's XGBoost.
- `param_distributions`: Unlike `params` - this is the distribution of possible hyperparameters to use.
- `cv`: Number of cross-validation iterations
- `n_iter`: Number of hyperparameter combinations to choose from
- `verbose`: Prints more output

In [58]:
from sklearn.model_selection import RandomizedSearchCV

# Define a parameter grid
rs_param_grid = {
    # max_depth: values from 3 to 12
    'max_depth': list((range(3,12))),
    # alpha: values 0, .001, .01, .1
    'alpha': [0,0.001, 0.01,0.1,1],
    # subsample: values 0.25,0.5,0.75, 1
    'subsample': [0.5,0.75,1],
    # learning rate: ten values between 0.01 - 0.5
    'learning_rate': np.linspace(0.01,0.5, 10),
    # n_estimators: values 10, 25, 40
    'n_estimators': [10, 25, 40]
    }


# Insantiate XGBoost Clasifier 
xgb_clf = xgb.XGBClassifier(random_state=123)

# Instantiate RandomizedSearchCV()
xgb_rs = RandomizedSearchCV(estimator=xgb_clf,param_distributions=rs_param_grid, 
                                cv=3, n_iter=5, verbose=2, random_state=123)

# Train the model on the training set
xgb_rs.fit(X_train, y_train)

# Print the best parameters and highest accuracy
print("Best parameters found: ", xgb_rs.best_params_)
print("Best accuracy found: ", xgb_rs.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] subsample=0.5, n_estimators=10, max_depth=5, learning_rate=0.22777777777777777, alpha=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.5, n_estimators=10, max_depth=5, learning_rate=0.22777777777777777, alpha=1, total=   2.5s
[CV] subsample=0.5, n_estimators=10, max_depth=5, learning_rate=0.22777777777777777, alpha=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV]  subsample=0.5, n_estimators=10, max_depth=5, learning_rate=0.22777777777777777, alpha=1, total=   2.3s
[CV] subsample=0.5, n_estimators=10, max_depth=5, learning_rate=0.22777777777777777, alpha=1 
[CV]  subsample=0.5, n_estimators=10, max_depth=5, learning_rate=0.22777777777777777, alpha=1, total=   2.4s
[CV] subsample=1, n_estimators=40, max_depth=6, learning_rate=0.11888888888888888, alpha=1 
[CV]  subsample=1, n_estimators=40, max_depth=6, learning_rate=0.11888888888888888, alpha=1, total=  10.8s
[CV] subsample=1, n_estimators=40, max_depth=6, learning_rate=0.11888888888888888, alpha=1 
[CV]  subsample=1, n_estimators=40, max_depth=6, learning_rate=0.11888888888888888, alpha=1, total=  11.6s
[CV] subsample=1, n_estimators=40, max_depth=6, learning_rate=0.11888888888888888, alpha=1 
[CV]  subsample=1, n_estimators=40, max_depth=6, learning_rate=0.11888888888888888, alpha=1, total=  11.2s
[CV] subsample=0.75, n_estimators=40, max_depth=8, learning_rate=0.11888888888888888, alpha

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=123, reg_alpha...
                   param_distributions={'alpha': [0, 0.001, 0.01, 0.1, 1],
                                        'learning_rate': array([0.01      , 0.06444444, 0.11888889, 0.17333333, 0.22777778,
       0.28222222, 0.33

Best parameters found:  {'subsample': 0.5, 'n_estimators': 40, 'max_depth': 10, 'learning_rate': 0.17333333333333334, 'alpha': 0.001}
Best accuracy found:  0.8360210341805434


---
<center><h1> Assignment </h1> </center>

---

## **Assignment**

What is the highest accuracy you can reach on the test set (`X_test`,`y_test`) after training on the training set (`X_train`,`y_train`)? Make sure to play around with the parameters and their values in `rs_param_grid`.

In [36]:
# Read in dataset
bookings= pd.read_csv('D:/Data/Northeastern University/Applications of AI/Module 3/bookings.csv', index_col = "Unnamed: 0")
pd.set_option('display.max_columns', None)

In [38]:
#Get the dimension of the dataset
bookings.shape
bookings.info()

(119210, 53)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119210 entries, 0 to 119209
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   is_canceled                     119210 non-null  int64  
 1   lead_time                       119210 non-null  int64  
 2   arrival_date_week_number        119210 non-null  int64  
 3   arrival_date_day_of_month       119210 non-null  int64  
 4   arrival_date_month              119210 non-null  int64  
 5   stays_in_weekend_nights         119210 non-null  int64  
 6   stays_in_week_nights            119210 non-null  int64  
 7   adults                          119210 non-null  int64  
 8   children                        119206 non-null  float64
 9   babies                          119210 non-null  int64  
 10  is_repeated_guest               119210 non-null  int64  
 11  previous_cancellations          119210 non-null  int64  
 12  previous_booking

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns


In [40]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [41]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost import XGBClassifier

In [42]:
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [43]:
# Read in dataset
bookings= pd.read_csv('D:/Data/Northeastern University/Applications of AI/Module 3/bookings.csv', index_col = "Unnamed: 0")
pd.set_option('display.max_columns', None)

In [44]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119210 entries, 0 to 119209
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   is_canceled                     119210 non-null  int64  
 1   lead_time                       119210 non-null  int64  
 2   arrival_date_week_number        119210 non-null  int64  
 3   arrival_date_day_of_month       119210 non-null  int64  
 4   arrival_date_month              119210 non-null  int64  
 5   stays_in_weekend_nights         119210 non-null  int64  
 6   stays_in_week_nights            119210 non-null  int64  
 7   adults                          119210 non-null  int64  
 8   children                        119206 non-null  float64
 9   babies                          119210 non-null  int64  
 10  is_repeated_guest               119210 non-null  int64  
 11  previous_cancellations          119210 non-null  int64  
 12  previous_booking

In [48]:
# Hotel types
plt.figure(figsize=(10,10))
sns.countplot(x='hotel_Resort', data = bookings, palette='viridis')
plt.title('Hotel Types', weight='bold')
plt.xlabel('hotel_Resort', fontsize=12)
plt.ylabel('Count', fontsize=12)

<Figure size 720x720 with 0 Axes>

<AxesSubplot:xlabel='hotel_Resort', ylabel='count'>

Text(0.5, 1.0, 'Hotel Types')

Text(0.5, 0, 'hotel_Resort')

Text(0, 0.5, 'Count')

In [50]:
# Display booking cancellation 
plt.figure(figsize=(10,10))
sns.countplot(y='is_canceled', data= bookings, palette='coolwarm', orient = 'v')
plt.title('Booking Cancellation', weight='bold')
plt.xlabel('Count', fontsize=12)
plt.ylabel('Canceled vs Not Canceled', fontsize=12)

<Figure size 720x720 with 0 Axes>

<AxesSubplot:xlabel='count', ylabel='is_canceled'>

Text(0.5, 1.0, 'Booking Cancellation')

Text(0.5, 0, 'Count')

Text(0, 0.5, 'Canceled vs Not Canceled')

In [51]:
# Display each year's distribution of booking cancellation status vs lead_time
plt.figure(figsize=(10,10))
sns.violinplot(x='arrival_date_month', y ='lead_time', hue="is_canceled", data=bookings, palette="coolwarm", bw=.2,
               cut=2, linewidth=2, iner= 'box', split = True)
sns.despine(left=True)
plt.title('Booking Cancellation Status vs Lead Time by Year', weight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Lead Time', fontsize=12)

<Figure size 720x720 with 0 Axes>

<AxesSubplot:xlabel='arrival_date_month', ylabel='lead_time'>

Text(0.5, 1.0, 'Booking Cancellation Status vs Lead Time by Year')

Text(0.5, 0, 'Month')

Text(0, 0.5, 'Lead Time')

In [52]:
# Convert `arrival_date_month` names to numbers
bookings['arrival_date_month'].replace({'January' : '1',
        'February' : '2',
        'March' : '3',
        'April' : '4',
        'May' : '5',
        'June' : '6',
        'July' : '7',
        'August' : '8',
        'September' : '9', 
        'October' : '10',
        'November' : '11',
        'December' : '12'}, inplace=True)

In [54]:
# Display arrival count by month 
plt.figure(figsize=(15,10))
sns.countplot(x='arrival_date_month', data = bookings,
              order=pd.value_counts(bookings['arrival_date_month']).index, palette='Greens_r')
plt.title('Arrival Month', weight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Count', fontsize=12)

<Figure size 1080x720 with 0 Axes>

<AxesSubplot:xlabel='arrival_date_month', ylabel='count'>

Text(0.5, 1.0, 'Arrival Month')

Text(0.5, 0, 'Month')

Text(0, 0.5, 'Count')

In [55]:
# Display relationship between `stay_in_weekend` & `stay_in_week_nights` columns
pd.crosstab(index = bookings['stays_in_week_nights'],columns=bookings['stays_in_weekend_nights'], margins=True, margins_name = 'Total').iloc[:10]

stays_in_weekend_nights,0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,18,19,Total
stays_in_week_nights,,,,,,,,,,,,,,,,,,
0,645,4569,2358,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7572
1,16436,7325,6531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30292
2,17949,8976,6745,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33670
3,11557,6150,4534,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22241
4,4478,2407,2658,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9543
5,830,1188,8648,308,94,0,0,0,0,0,0,0,0,0,0,0,0,11068
6,0,0,847,300,347,0,0,0,0,0,0,0,0,0,0,0,0,1494
7,0,0,446,397,181,0,0,0,0,0,0,0,0,0,0,0,0,1024
8,0,0,391,131,132,0,0,0,0,0,0,0,0,0,0,0,0,654


In [56]:
# Create new feature: 'weekend_or_weekday' 
pd.options.mode.chained_assignment = None
def weekendOrWeekday(feature1, feature2, data_source):
    data_source['weekend_or_weekday'] = 0
    for i in range(0, len(data_source)):
        if feature2.iloc[i] == 0 and feature1.iloc[i] > 0:
            bookings['weekend_or_weekday'].iloc[i] = 'weekend_only_stay'
        if feature2.iloc[i] > 0 and feature1.iloc[i] == 0:
            bookings['weekend_or_weekday'].iloc[i] = 'weekday_only_stay'
        if feature2.iloc[i] > 0 and feature1.iloc[i] > 0:
            bookings['weekend_or_weekday'].iloc[i] = 'weekday_and_weekend_stay'
        if feature2.iloc[i] == 0 and feature1.iloc[i] == 0:
            bookings['weekend_or_weekday'].iloc[i] = 'undefined_data'

            
weekendOrWeekday(bookings['stays_in_weekend_nights'],bookings['stays_in_week_nights'], bookings)

In [57]:
# Graph `arrival_date_month` vs `weekend_or_weekday` 
bookings['arrival_date_month']= bookings['arrival_date_month'].astype('int64')
groupData = bookings.groupby([ 'arrival_date_month','weekend_or_weekday']).size().unstack(fill_value=0)
groupData.sort_values('arrival_date_month', ascending = True).plot(kind='bar',stacked=True, cmap='Dark2_r',figsize=(15,10))
plt.title('Arrival Month vs Weekend or Weekday Stay', weight='bold')
plt.xlabel('Arrival Month', fontsize=12)
plt.xticks(rotation=360)
plt.ylabel('Bookings', fontsize=12)

<AxesSubplot:xlabel='arrival_date_month'>

Text(0.5, 1.0, 'Arrival Month vs Weekend or Weekday Stay')

Text(0.5, 0, 'Arrival Month')

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 [Text(0, 0, '1'),
  Text(1, 0, '2'),
  Text(2, 0, '3'),
  Text(3, 0, '4'),
  Text(4, 0, '5'),
  Text(5, 0, '6'),
  Text(6, 0, '7'),
  Text(7, 0, '8'),
  Text(8, 0, '9'),
  Text(9, 0, '10'),
  Text(10, 0, '11'),
  Text(11, 0, '12')])

Text(0, 0.5, 'Bookings')

In [58]:
# Create new feature:`all_children`
bookings['all_children'] = bookings['children'] + bookings['babies']
pd.crosstab(bookings['adults'], bookings['all_children'], margins=True, margins_name = 'Total').iloc[:10]

all_children,0.0,1.0,2.0,3.0,9.0,10.0,Total
adults,,,,,,,
0,0,4,205,14,0,0,223
1,22577,286,157,6,1,0,23027
2,81557,4658,3369,91,0,2,89677
3,5665,495,41,0,0,0,6201
4,59,3,0,0,0,0,62
5,2,0,0,0,0,0,2
6,1,0,0,0,0,0,1
10,1,0,0,0,0,0,1
20,2,0,0,0,0,0,2


In [61]:
bookings.isnull().sum()

is_canceled                       0
lead_time                         0
arrival_date_week_number          0
arrival_date_day_of_month         0
arrival_date_month                0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          4
babies                            0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
required_car_parking_spaces       0
total_of_special_requests         0
avg_daily_rate                    0
booked_by_company                 0
booked_by_agent                   0
hotel_City                        0
hotel_Resort                      0
meal_BB                           0
meal_FB                           0
meal_HB                           0
meal_No_meal                      0
market_segment_Aviation           0
market_segment_Complementary      0
market_segment_Corporate          0
market_segment_Direct       

In [65]:
# Fill in missing data
bookings['children'] =  bookings['children'].fillna(0)
bookings['all_children'] = bookings['all_children'].fillna(0)


In [80]:
labelencoder = LabelEncoder()
bookings['arrival_date_month'] = labelencoder.fit_transform(bookings['arrival_date_month'])
bookings['meal_BB'] = labelencoder.fit_transform(bookings['meal_BB'])
bookings['market_segment_Aviation']= labelencoder.fit_transform(bookings['market_segment_Aviation'])
bookings['distribution_channel_Corporate']=labelencoder.fit_transform(bookings['distribution_channel_Corporate'])
bookings['is_repeated_guest'] = labelencoder.fit_transform(bookings['is_repeated_guest'])
bookings['reserved_room_type_A'] = labelencoder.fit_transform(bookings['reserved_room_type_A'])
bookings['deposit_type_Refundable'] = labelencoder.fit_transform(bookings['deposit_type_Refundable'])
bookings['customer_type_Contract'] = labelencoder.fit_transform(bookings['customer_type_Contract'])
bookings['weekend_or_weekday'] = labelencoder.fit_transform(bookings['weekend_or_weekday'])

In [83]:
# Create new DataFrame for categorical data
hotelDataCategorical = bookings[['is_canceled','arrival_date_month','meal_BB',
                                     'market_segment_Aviation','distribution_channel_Corporate', 
                                     'is_repeated_guest', 'reserved_room_type_A',
                                     'deposit_type_Refundable',
                                     'customer_type_Contract', 
                                     'weekend_or_weekday']]

In [84]:
hotelDataCategorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119210 entries, 0 to 119209
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype
---  ------                          --------------   -----
 0   is_canceled                     119210 non-null  int64
 1   arrival_date_month              119210 non-null  int64
 2   meal_BB                         119210 non-null  int64
 3   market_segment_Aviation         119210 non-null  int64
 4   distribution_channel_Corporate  119210 non-null  int64
 5   is_repeated_guest               119210 non-null  int64
 6   reserved_room_type_A            119210 non-null  int64
 7   deposit_type_Refundable         119210 non-null  int64
 8   customer_type_Contract          119210 non-null  int64
 9   weekend_or_weekday              119210 non-null  int32
dtypes: int32(1), int64(9)
memory usage: 13.6 MB


In [85]:
hotelDataNumerical= bookings.drop(['is_canceled','arrival_date_month','meal_BB',
                                     'market_segment_Aviation','distribution_channel_Corporate', 
                                     'is_repeated_guest', 'reserved_room_type_A',
                                     'deposit_type_Refundable',
                                     'customer_type_Contract', 
                                     'weekend_or_weekday'], axis = 1)

In [86]:
hotelDataNumerical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119210 entries, 0 to 119209
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   lead_time                       119210 non-null  int64  
 1   arrival_date_week_number        119210 non-null  int64  
 2   arrival_date_day_of_month       119210 non-null  int64  
 3   stays_in_weekend_nights         119210 non-null  int64  
 4   stays_in_week_nights            119210 non-null  int64  
 5   adults                          119210 non-null  int64  
 6   children                        119210 non-null  float64
 7   babies                          119210 non-null  int64  
 8   previous_cancellations          119210 non-null  int64  
 9   previous_bookings_not_canceled  119210 non-null  int64  
 10  required_car_parking_spaces     119210 non-null  int64  
 11  total_of_special_requests       119210 non-null  int64  
 12  avg_daily_rate  

In [87]:
# Seperate target variable
hotelDataTunning = bookings
y = hotelDataTunning.iloc[:,1]
X = pd.concat([hotelDataTunning.iloc[:,0],hotelDataTunning.iloc[:,2:30]], axis=1)

In [89]:
## Finding parameters for XGBoost model

model = XGBClassifier()
parameters = {
'n_estimators' : [100,250,500],
 'learning_rate' : [0.01, 0.1],
 'subsample' :[0.5, 1.0],
 'max_depth' : [3,5,7],
 'criterion' : ['giny','entropy'],
 'objective':['binary:logistic'],
 }

 grid_search = GridSearchCV(estimator=model, param_grid=parameters,
                           cv=5, scoring='f1', verbose=True, n_jobs=-1)
 grid_search.fit(X, y)
 print(grid_search.best_score_)
 print(grid_search.best_params_)

IndentationError: unexpected indent (<ipython-input-89-682b5c267256>, line 13)

In [93]:
# Permutation Importance graph with XGB Classifier algorithm
params = {
    'criterion': 'giny', 
    'learning_rate': 0.01, 
    'max_depth': 5,
    'n_estimators': 100, 
    'objective': 'binary:logistic', 
}
model = XGBClassifier(parameters=params)
# fit the model
model.fit(X, y)
# perform permutation importance
result = permutation_importance(model, X, y, scoring='accuracy', n_repeats = 5, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()

E:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:57:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { parameters } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:57:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob',
              parameters={'criterion': 'giny', 'learning_rate': 0.01,
                          'max_depth': 5, 'n_estimators': 100,
                          'objective': 'binary:logistic'},
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

XGBoostError: bad allocation

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "E:\Anaconda\lib\site-packages\joblib\externals\loky\backend\queues.py", line 150, in _feed
    obj_ = dumps(obj, reducers=reducers)
  File "E:\Anaconda\lib\site-packages\joblib\externals\loky\backend\reduction.py", line 247, in dumps
    dump(obj, buf, reducers=reducers, protocol=protocol)
  File "E:\Anaconda\lib\site-packages\joblib\externals\loky\backend\reduction.py", line 240, in dump
    _LokyPickler(file, reducers=reducers, protocol=protocol).dump(obj)
  File "E:\Anaconda\lib\site-packages\joblib\externals\cloudpickle\cloudpickle.py", line 482, in dump
    return Pickler.dump(self, obj)
  File "E:\Anaconda\lib\pickle.py", line 437, in dump
    self.save(obj)
  File "E:\Anaconda\lib\pickle.py", line 549, in save
    self.save_reduce(obj=obj, *rv)
  File "E:\Anaconda\lib\pickle.py", line 662, in save_reduce
    save(state)
  File "E:\Anaconda\lib\pickle.py", line 504, in save
    f(self, obj) # Call u

In [94]:
for i,v in enumerate(sortedIdx):
    print('Feature: %0d, Score: %.5f' % (i,v))

NameError: name 'sortedIdx' is not defined

In [103]:
# Seperate target variable for model building 
hotel_data_model = bookings
hotel_data_model = hotel_data_model.drop(['babies'], axis=1)
y_model = hotel_data_model.iloc[:,1]
X_model = pd.concat([hotelDataTunning.iloc[:,0],hotelDataTunning.iloc[:,2:30]], axis=1)
y_model.describe()

count    119210.000000
mean        104.109227
std         106.875450
min           0.000000
25%          18.000000
50%          69.000000
75%         161.000000
max         737.000000
Name: lead_time, dtype: float64

In [106]:
# Split to train and test with 70-30 ratio
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.3, random_state=30, stratify = y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [107]:
# Use standard scaler
standardScalerX = StandardScaler()
X_train = standardScalerX.fit_transform(X_train)
X_test = standardScalerX.fit_transform(X_test)

In [108]:
# Extreme Gradient Boosting Model Building
xgb_model = XGBClassifier(criterion = 'giny', learning_rate = 0.01, max_depth = 5, n_estimators = 100,
                          objective ='binary:logistic', subsample = 1.0)
# Fit the model
xgb_model.fit(X_train, y_train)
# Model prediction
predict_xgb = xgb_model.predict(X_test)

E:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { criterion } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:59:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, criterion='giny', gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1.0,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [109]:
print("XGB", classification_report(y_test, predict_xgb))

XGB               precision    recall  f1-score   support

           0       0.80      0.93      0.86     24627
           1       0.84      0.60      0.70     14713

    accuracy                           0.81     39340
   macro avg       0.82      0.77      0.78     39340
weighted avg       0.81      0.81      0.80     39340

